# Anomaly Detection for UNSW NB-15 cyberattack dataset
Identifying cyberattacks can be considered both a classification problem and an anomaly detection problem. In this notebook, I treat it as an anomaly detection problem. Given that [the training and test csv's provided by the researchers](https://cloudstor.aarnet.edu.au/plus/index.php/s/2DhnLGDdEECo4ys?path=%2FUNSW-NB15%20-%20CSV%20Files%2Fa%20part%20of%20training%20and%20testing%20set) are balanced between the normal and anomaly classes, I'm going to down-sample to reduce the available number of anomalies.

Like the notebook `02_basic_classification`, this notebook is a first-pass attempt with serious flaws. It is meant to ensure that I can actually run models on the dataset's numeric columns as is, and to get a familiarity with how these models work as implemented in Scikit-Learn.

In [45]:
# Custom modules
from data_prep import load_csv_data, y_anomaly_format 
import model_abstraction as moda

# Data Structures
import pandas as pd
import numpy as np

# Modeling methods and selection
from sklearn.model_selection import train_test_split

# Anomaly detection
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor

# Model assessment
from sklearn.metrics import confusion_matrix, roc_auc_score

# Visualization if needed
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Modeling Using only Numeric Features
The columns that remain after excluding string objects are all numerical, but contain a mix of ordinal, categorical, integer, and float values. I'm going to run the dataset through a selection of outlier detection methods available in Scikit-Learn with the following strategy:
* train on normal examples
* predict on normal examples in training set
* predict on outlier examples in training set
* predict on training set as a whole
* predict on holdout set as a whole

In [40]:
# The function below loads in the training set with only non-numeric features.
# It uses a binary label (-1 for outliers, 1 for inliers) that doesn't distinguish between types of attacks.
X_train, y_train = load_csv_data('./data/UNSW_NB15_train_set.csv', strategy='anomaly')
X_train, X_hold, y_train, y_hold  = train_test_split(X_train,y_train, test_size = 0.25,
                                                     random_state = 42, stratify=y_train)
X_test, y_test = load_csv_data('./data/UNSW_NB15_test_set.csv', strategy='anomaly')

In [4]:
# Number of numeric features remaining
len(X_train.columns)

39

In [5]:
y_train.unique()

array([ 1, -1])

The type of anomaly detection methods used below start by training on normal data and building a profile. From there, they can be used to predict on unseen data. The `train_normal` mask I create below is used throughout the notebook to quickly separate normal data from attack data.

In [6]:
## Create 'masks' to filter the dataframe by whether or not the observation belongs to normal or attack class.
train_normal = y_train==1

## Isolation Forest

In [11]:
ilf = IsolationForest()
ilf.fit(X_train[train_normal])


/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:214: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:224: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:418: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:418: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


In [13]:
y_pred_nm = ilf.predict(X_train[train_normal])
y_pred_out = ilf.predict(X_train[~train_normal])
y_pred_train = ilf.predict(X_train)
y_pred_test = ilf.predict(X_test)


/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:418: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:418: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:418: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:418: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


In [17]:
confusion_matrix(y_train[train_normal], y_pred_nm)

array([[    0,     0],
       [ 5600, 50400]])

In [16]:
confusion_matrix(y_train[~train_normal], y_pred_out)

array([[54217, 65124],
       [    0,     0]])

In [18]:
confusion_matrix(y_train, y_pred_train)

array([[54217, 65124],
       [ 5600, 50400]])

In [38]:
print(roc_auc_score(y_test, y_pred_test))
confusion_matrix(y_test, y_pred_test)

0.6193118243541345


array([[17214, 28118],
       [ 5221, 31779]])

In [22]:
y_test.value_counts()

-1    45332
 1    37000
Name: label, dtype: int64

## One Class SVM

In [27]:
ocsvm = OneClassSVM(kernel='rbf')
ocsvm.fit(X_train[train_normal])
y_pred_normal = ocsvm.fit_predict(X_train[train_normal])
y_pred_outlier = ocsvm.predict(X_train[~train_normal])
y_pred = ocsvm.predict(X_train)

/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [28]:
confusion_matrix(y_train[train_normal], y_pred_normal)

array([[    0,     0],
       [26719, 29281]])

In [29]:
confusion_matrix(y_train[~train_normal], y_pred_outlier)

array([[117839,   1502],
       [     0,      0]])

In [30]:
confusion_matrix(y_train, y_pred)

array([[117839,   1502],
       [ 26719,  29281]])

In [35]:
roc_auc_score(y_test, ocsvm.predict(X_test))

0.551775488229781

## Elliptic Envelope
Elliptic envelope works by forming a "shape" around the normal data. This is based on the assumption that the data comes from some known distribution around which a boundary can be fit. Observations that then fall far enough away from the shape are marked as outliers.

In [25]:
eenv = EllipticEnvelope(contamination=0.2, random_state=0)
eenv.fit(X_train[train_normal])

/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:646: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (166.386025488722566 > 127.388028898191905). You may want to try with a higher value of support_fraction (current value: 0.502).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (176.328360669299229 > 156.253386375134369). You may want to try with a higher value of support_fraction (current value: 0.502).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has

/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (186.839118631325164 > 127.564683925236125). You may want to try with a higher value of support_fraction (current value: 0.502).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (166.250117545521675 > 146.120374932246932). You may want to try with a higher value of support_fraction (current value: 0.502).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (207.472373462294428 > 142.908096661021716). You may want to try with a higher value of support_fraction (current value: 0.502).
  RuntimeWarning)
/anaconda3/lib/pyth

/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (178.421507943189823 > 131.972477587470451). You may want to try with a higher value of support_fraction (current value: 0.502).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (158.173034280833406 > 148.139030120393045). You may want to try with a higher value of support_fraction (current value: 0.502).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (151.775074119897141 > 149.074020918495279). You may want to try with a higher value of support_fraction (current value: 0.502).
  RuntimeWarning)
/anaconda3/lib/pyth

/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (168.471279770044447 > 159.149105256651666). You may want to try with a higher value of support_fraction (current value: 0.502).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (182.325130612578562 > 139.672067527606288). You may want to try with a higher value of support_fraction (current value: 0.502).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (183.283413536762737 > 176.964569029750805). You may want to try with a higher value of support_fraction (current value: 0.502).
  RuntimeWarning)
/anaconda3/lib/pyth

/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (201.545422270982868 > 182.661557969439770). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (207.233703998092778 > 186.865938403371104). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (200.658496475770761 > 194.274686335393937). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/anaconda3/lib/pyth

...

/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (230.625216546927845 > 229.592975836933192). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (212.014693804501832 > 201.917735702289207). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (187.603560445388780 > 185.370530946522734). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/anaconda3/lib/pyth

/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (227.080679502010241 > 206.599592741978228). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (228.004905123247937 > 169.249119455880674). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (225.767452173648110 > 187.475766153075369). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/anaconda3/lib/pyth

/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (241.619169309112976 > 197.893948791708453). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (191.353309950036703 > 190.946469695377630). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (242.125821859055293 > 188.649595079627318). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/anaconda3/lib/pyth

/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (214.306314163444483 > 169.675348795614667). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (244.748702789148922 > 220.357424952830854). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (207.167966651481493 > 177.023549208693169). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/anaconda3/lib/pyth

EllipticEnvelope(assume_centered=False, contamination=0.2, random_state=0,
         store_precision=True, support_fraction=None)

In [36]:
roc_auc_score(y_test,eenv.predict(X_test))

0.7075123032235447

## Local Outlier Factor

In [59]:
lof = LocalOutlierFactor()
y_pred = lof.fit_predict(X_train)

/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


In [43]:
neg_scores = lof.negative_outlier_factor_
out_scores = (neg_scores.max()-neg_scores)/(neg_scores.max()-neg_scores.min())

In [53]:
pd.Series(neg_scores).describe()

count    1.315050e+05
mean    -7.703878e+12
std      2.633116e+14
min     -2.320057e+16
25%     -1.078605e+00
50%     -1.004495e+00
75%     -1.000000e+00
max     -8.769621e-01
dtype: float64

In [54]:
for lim in np.linspace(1e-5,1e1,10):
    print(roc_auc_score(y_train, list(map(y_anomaly_format, out_scores > lim))))

0.5005816494689044
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5


In [61]:
roc_auc_score(y_train, y_pred)

0.5393576376419513